In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import copy

# 1. Import Dataset

In [2]:
DATA_FOLDER_NAME = 'data'

sampleOrigFiles = str(os.getcwd())+"/" + DATA_FOLDER_NAME + "\sample_orig_*.txt"
outputOrigFile = 'SampleOriginationCombined.csv'

orig_headers = ['CREDIT_SCORE','FIRST_PAYMENT_DATE','FIRST_TIME_HOMEBUYER_FLAG','MATURITY_DATE','MSA','MI_PCT',
                'NUMBER_OF_UNITS','OCCUPANCY_STATUS','CLTV','DTI','ORIGINAL_UPB','LTV','ORIGINAL_INTEREST_RATE',
                'CHANNEL','PPM','AMORTIZATION_TYPE','PROPERTY_STATE', 'PROPERTY_TYPE','POSTAL_CODE',
                'LOAN_SEQUENCE_NUMBER','LOAN_PURPOSE', 'ORIGINAL_LOAN_TERM','NUMBER_OF_BORROWERS','SELLER_NAME',
                'SERVICER_NAME','SUPER_CONFORMING_FLAG','PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER', 
                'PROGRAM_INDICATOR', 'RELIEF_REFINANCE_INDICATOR', 'PROPERTY_VALUATION_METHOD', 'IO_INDICATOR']

raw_orig_types = {'CREDIT_SCORE':float, 'FIRST_PAYMENT_DATE':str, 'FIRST_TIME_HOMEBUYER_FLAG':str, 'MATURITY_DATE':str,
              'MSA':str, 'MI_PCT':float, 'NUMBER_OF_UNITS':float, 'OCCUPANCY_STATUS':str, 'CLTV':float, 'DTI':float,
              'ORIGINAL_UPB':float, 'LTV':float, 'ORIGINAL_INTEREST_RATE':float, 'CHANNEL':str, 'PPM': str,
              'AMORTIZATION_TYPE':str ,'PROPERTY_STATE':str, 'PROPERTY_TYPE':str ,'POSTAL_CODE':str,
              'LOAN_SEQUENCE_NUMBER':str, 'LOAN_PURPOSE':str, 'ORIGINAL_LOAN_TERM':float, 'NUMBER_OF_BORROWERS':float,
              'SELLER_NAME': str, 'SERVICER_NAME':str, 'SUPER_CONFORMING_FLAG':str,
              'PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER':str, 'PROGRAM_INDICATOR':str, 'RELIEF_REFINANCE_INDICATOR':str,
              'PROPERTY_VALUATION_METHOD':str, 'IO_INDICATOR':str}

mutated_orig_types = raw_orig_types.copy()

orig_dates = ['FIRST_PAYMENT_DATE', 'MATURITY_DATE']

samplePerfFiles = str(os.getcwd())+"/" + DATA_FOLDER_NAME + "\sample_svcg_*.txt"
outputPerfFile = 'SamplePerformanceCombined.csv'

perf_headers = ['LOAN_SEQUENCE_NUMBER','MONTHLY_REPORTING_PERIOD','CURRENT_ACTUAL_UPB',
                'CURRENT_LOAN_DELINQUENCY_STATUS','LOAN_AGE','REMAINING_MONTHS_TO_LEGAL_MATURITY', 
                'DEFECT_SETTLEMENT_DATE','MODIFICATION_FLAG', 'ZERO_BALANCE_CODE', 
                'ZERO_BALANCE_EFFECTIVE_DATE','CURRENT_INTEREST_RATE','CURRENT_DEFERRED_UPB','DDLPI',
                'MI_RECOVERIES', 'NET_SALE_PROCEEDS','NON_MI_RECOVERIES','EXPENSES', 'LEGAL_COSTS',
                'MAINTENANCE_AND_PRESERVATION_COSTS','TAXES_AND_INSURANCE','MISCELLANEOUS_EXPENSES',
                'ACTUAL_LOSS_CALCULATION', 'MODIFICATION_COST','STEP_MODIFICATION_FLAG','DEFERRED_PAYMENT_PLAN',
                'ELTV','ZERO_BALANCE_REMOVAL_UPB','DELINQUENT_ACCRUED_INTEREST','DELINQUENCY_DUE_TO_DISASTER',
                'BORROWER_ASSISTANCE_STATUS_CODE','CURRENT_MONTH_MODIFICATION_COST','INTEREST_BEARING_UPB']

raw_perf_types = {'LOAN_SEQUENCE_NUMBER':str, 'MONTHLY_REPORTING_PERIOD':str, 'CURRENT_ACTUAL_UPB':float,
              'CURRENT_LOAN_DELINQUENCY_STATUS': str, 'LOAN_AGE':float, 'REMAINING_MONTHS_TO_LEGAL_MATURITY':float, 
              'DEFECT_SETTLEMENT_DATE':str, 'MODIFICATION_FLAG':str, 'ZERO_BALANCE_CODE':str,
              'ZERO_BALANCE_EFFECTIVE_DATE':str, 'CURRENT_INTEREST_RATE':float, 'CURRENT_DEFERRED_UPB':float,
              'DDLPI': str,'MI_RECOVERIES':float, 'NET_SALE_PROCEEDS':str ,'NON_MI_RECOVERIES':float,'EXPENSES':float,
              'LEGAL_COSTS':float ,'MAINTENANCE_AND_PRESERVATION_COSTS':float, 'TAXES_AND_INSURANCE':float,
              'MISCELLANEOUS_EXPENSES':float,'ACTUAL_LOSS_CALCULATION':float, 'MODIFICATION_COST':float,
              'STEP_MODIFICATION_FLAG':str, 'DEFERRED_PAYMENT_PLAN':str, 'ELTV':float, 'ZERO_BALANCE_REMOVAL_UPB':float,
              'DELINQUENT_ACCRUED_INTEREST':float, 'DELINQUENCY_DUE_TO_DISASTER':str,'BORROWER_ASSISTANCE_STATUS_CODE':str,
              'CURRENT_MONTH_MODIFICATION_COST':float, 'INTEREST_BEARING_UPB':float}

mutated_perf_types = raw_perf_types.copy()
mutated_perf_types['CURRENT_LOAN_DELINQUENCY_STATUS'] = float



perf_dates = ['DEFECT_SETTLEMENT_DATE', 'ZERO_BALANCE_EFFECTIVE_DATE', 'DDLPI']

In [3]:
from FREDDIEMAC_preprocessing import create_origination_from_raw_data, create_performance_from_raw_data

create_origination_from_raw_data(sampleOrigFiles, orig_headers, raw_orig_types, outputOrigFile)
create_performance_from_raw_data(samplePerfFiles, perf_headers, raw_perf_types, outputPerfFile)

df_orig = pd.read_csv(outputOrigFile, dtype=mutated_orig_types)
df_perf = pd.read_csv(outputPerfFile, dtype=mutated_perf_types)

Working on  c:\Users\marij\Desktop\THESIS ECO\datasets\FREDDIEMAC/data\sample_orig_2010.txt: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]
Working on  c:\Users\marij\Desktop\THESIS ECO\datasets\FREDDIEMAC/data\sample_svcg_2010.txt: 100%|██████████| 1/1 [01:24<00:00, 84.39s/it]


In [4]:
from FREDDIEMAC_classification import assign_labels_to_orig, assign_labels_to_perf, cutoff_sequence_according_to_label
from FREDDIEMAC_reducing import drop_short_sequences

df_orig = assign_labels_to_orig(df_orig, df_perf)
df_perf = assign_labels_to_perf(df_orig, df_perf)
df_orig, df_perf = cutoff_sequence_according_to_label(df_orig, df_perf)
df_orig, df_perf = drop_short_sequences(df_orig, df_perf, 12)

In [5]:
df_orig

,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,MSA,MI_PCT,NUMBER_OF_UNITS,OCCUPANCY_STATUS,CLTV,DTI,...,IO_INDICATOR,QUARTER,YEAR,DEFAULT,PREPAYMENT_OR_MATURED,ORIGINAL_REMAINING_MONTHS_TO_LEGAL_MATURITY,LABEL,REMAINING_MONTHS_TO_LEGAL_MATURITY,TOTAL_OBSERVED_LENGTH,TIME_TO_EVENT
0,784.0,201005,N,202504,45780,0.0,1.0,P,90.0,38.0,...,N,1,10,0.0,1.0,155.0,0,156.0,25.0,25.0
1,795.0,201003,N,202502,0,0.0,1.0,P,67.0,35.0,...,N,1,10,0.0,1.0,155.0,0,156.0,25.0,25.0
2,784.0,201003,N,202502,36100,0.0,1.0,P,55.0,47.0,...,N,1,10,0.0,1.0,119.0,0,120.0,61.0,61.0
3,759.0,201003,N,204002,47260,0.0,1.0,P,50.0,27.0,...,N,1,10,0.0,1.0,226.0,0,227.0,134.0,134.0
4,695.0,201004,N,204003,0,0.0,1.0,P,50.0,21.0,...,N,1,10,0.0,1.0,291.0,0,291.0,115.0,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,750.0,201102,Y,204101,0,0.0,1.0,P,97.0,28.0,...,N,4,10,0.0,1.0,241.0,0,242.0,33.0,33.0
49995,799.0,201012,Y,204011,35840,0.0,1.0,P,87.0,25.0,...,N,4,10,0.0,0.0,227.0,3,227.0,46.0,46.0
49997,717.0,201102,Y,204101,23224,0.0,1.0,P,95.0,22.0,...,N,4,10,0.0,1.0,247.0,0,248.0,21.0,21.0
49998,765.0,201111,N,204110,25940,0.0,1.0,P,75.0,42.0,...,N,4,10,0.0,1.0,257.0,0,258.0,20.0,20.0


In [6]:
df_perf

,LOAN_SEQUENCE_NUMBER,MONTHLY_REPORTING_PERIOD,CURRENT_ACTUAL_UPB,CURRENT_LOAN_DELINQUENCY_STATUS,LOAN_AGE,REMAINING_MONTHS_TO_LEGAL_MATURITY,DEFECT_SETTLEMENT_DATE,MODIFICATION_FLAG,ZERO_BALANCE_CODE,ZERO_BALANCE_EFFECTIVE_DATE,...,INTEREST_BEARING_UPB,QUARTER,YEAR,REO_ACQUISITION,ORIGINAL_TOTAL_OBSERVED_LENGTH,DEFAULT,PREPAYMENT_OR_MATURED,LABEL,TOTAL_OBSERVED_LENGTH,TIME_TO_EVENT
6,0000014,201004,216000.00,0.0,0.0,180.0,X,N,0,189901,...,216000.00,1,10,0,26,0,0,0,25,25
7,0000014,201005,215000.00,0.0,1.0,179.0,X,N,0,189901,...,215000.00,1,10,0,26,0,0,0,25,25
8,0000014,201006,214000.00,0.0,2.0,178.0,X,N,0,189901,...,214000.00,1,10,0,26,0,0,0,25,25
9,0000014,201007,213000.00,0.0,3.0,177.0,X,N,0,189901,...,213000.00,1,10,0,26,0,0,0,25,25
10,0000014,201008,213000.00,0.0,4.0,176.0,X,N,0,189901,...,213000.00,1,10,0,26,0,0,0,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3338820,0595385,202007,211529.84,0.0,114.0,251.0,X,N,0,189901,...,211529.84,4,10,0,28,0,0,0,27,27
3338821,0595385,202008,211529.84,0.0,115.0,250.0,X,N,0,189901,...,211529.84,4,10,0,28,0,0,0,27,27
3338822,0595385,202009,210486.02,0.0,116.0,249.0,X,N,0,189901,...,210486.02,4,10,0,28,0,0,0,27,27
3338823,0595385,202010,210486.02,0.0,117.0,248.0,X,N,0,189901,...,210486.02,4,10,0,28,0,0,0,27,27


### 1.1 Data Exploration

In [ ]:
df_perf['LOAN_SEQUENCE_NUMBER'].nunique()

In [ ]:
df_orig['LOAN_SEQUENCE_NUMBER'].nunique()

# 2. Aggregating data like Blumenstock et al. 2020 experiment 4.1

In [8]:
from FREDDIEMAC_reducing import reduce_length_of_sequence
from FREDDIEMAC_aggregation import aggregate_to_blumenstock_exp4

df_perf_concatenated = reduce_length_of_sequence(df_perf, length=12)
df_perf_concatenated.to_csv("df_perf_concatenated.csv", header=True, index=False)


In [9]:
df_perf_concatenated = pd.read_csv("df_perf_concatenated.csv", dtype=mutated_perf_types)

In [10]:
df_blumenstock = aggregate_to_blumenstock_exp4(df_orig, df_perf_concatenated)


In [11]:
df_blumenstock.to_csv("df_blumenstock.csv", header=True, index=False)

In [12]:
blumenstock_types = {'LOAN_SEQUENCE_NUMBER': str, 'ORIGINAL_INTEREST_RATE': float, 'ORIGINAL_UPB': float, 'CREDIT_SCORE': float,
                    'DTI': float, 'LTV': float, 'FIRST_PAYMENT_DATE': str, 'BAL_REPAID': float, 'T_ACT_12M': float, 'T_DEL_30D': float, 
                    'T_DEL_60D': float, 'LABEL': str, 'REMAINING_MONTHS_TO_LEGAL_MATURITY': float, 'TOTAL_OBSERVED_LENGTH': float}

df_blumenstock = pd.read_csv("df_blumenstock.csv", dtype=blumenstock_types)

In [13]:
df_blumenstock

,LOAN_SEQUENCE_NUMBER,ORIGINAL_INTEREST_RATE,ORIGINAL_UPB,CREDIT_SCORE,DTI,LTV,FIRST_PAYMENT_DATE,REMAINING_MONTHS_TO_LEGAL_MATURITY,TOTAL_OBSERVED_LENGTH,TIME_TO_EVENT,BAL_REPAID,T_ACT_12M,T_DEL_30D,T_DEL_60D
0,0000014,4.375,216000.0,784.0,38.0,80.0,201005,156.0,25.0,25.0,0.901226,12.0,0.0,0.0
1,0000069,4.500,200000.0,795.0,35.0,67.0,201003,156.0,25.0,25.0,0.836209,12.0,0.0,0.0
2,0000089,4.500,146000.0,784.0,47.0,55.0,201003,120.0,61.0,61.0,0.724131,12.0,0.0,0.0
3,0000098,4.875,130000.0,759.0,27.0,50.0,201003,227.0,134.0,134.0,0.638864,12.0,0.0,0.0
4,0000102,5.000,40000.0,695.0,21.0,50.0,201004,291.0,115.0,115.0,0.000000,12.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47125,0595140,3.875,160000.0,750.0,28.0,97.0,201102,242.0,33.0,33.0,0.788220,12.0,0.0,0.0
47126,0595181,4.250,107000.0,799.0,25.0,87.0,201012,227.0,46.0,46.0,0.599778,12.0,0.0,0.0
47127,0595364,4.250,361000.0,717.0,22.0,95.0,201102,248.0,21.0,21.0,0.800266,12.0,0.0,0.0
47128,0595368,4.375,353000.0,765.0,42.0,75.0,201111,258.0,20.0,20.0,0.824841,12.0,0.0,0.0


In [ ]:
df_blumenstock['LABEL'].value_counts() / df_blumenstock['LABEL'].value_counts().sum()

In [ ]:
df_blumenstock[df_blumenstock['LABEL'] == "0"]

In [ ]:
df_perf[df_perf['LOAN_SEQUENCE_NUMBER'] == "0000014"]

In [ ]:
df_perf_concatenated[(df_perf_concatenated['CURRENT_ACTUAL_UPB'] == 0) & (df_perf_concatenated['ZERO_BALANCE_CODE'] != "01")]

In [ ]:
df_perf['REMAINING_MONTHS_TO_LEGAL_MATURITY'].min()

In [ ]:
df_perf.loc[(df_perf['ZERO_BALANCE_CODE'] == "01") & (df_perf['REMAINING_MONTHS_TO_LEGAL_MATURITY'] == 0)].head(25)

# 3. Expanding Blumenstock et al. 2020 experiment 4.1 to dynamic data

In [ ]:
df_dynamic_blumenstock = aggregate_to_blumenstock_exp4(df_orig, df_perf_concatenated)